In [18]:
import h5py
import os
import numpy as np
import pandas as pd

In [19]:
os.chdir('/Users/sunnyyou/Documents/Real_Time_HAI/HPCNN/H5/')

In [31]:
f = h5py.File('3B-HHR-E.MS.MRG.3IMERG.20230907-S173000-E175959.1050.V06D.RT-H5', 'r')
print(f.keys())

<KeysViewHDF5 ['Grid']>


In [32]:
grid = f['Grid']
print(grid.keys())

<KeysViewHDF5 ['nv', 'lonv', 'latv', 'time', 'lon', 'lat', 'time_bnds', 'lon_bnds', 'lat_bnds', 'precipitationCal', 'precipitationUncal', 'randomError', 'HQprecipitation', 'HQprecipSource', 'HQobservationTime', 'IRprecipitation', 'IRkalmanFilterWeight', 'probabilityLiquidPrecipitation', 'precipitationQualityIndex']>


In [33]:
print(grid['lon'])

<HDF5 dataset "lon": shape (3600,), type "<f4">


In [34]:
max(list(grid['lat']))

89.95

In [35]:
longitude_values = np.repeat(list(grid['lon']), 1800)
latitude_values = list(grid['lat'])*3600
precipitation_values = np.array(list(grid['precipitationCal'])).flatten()

dataset = pd.DataFrame({"lon": longitude_values, "lat": latitude_values, "precipitation": precipitation_values})
dataset.columns = [grid['lon'].attrs['standard_name'].decode() + " (" + grid['lon'].attrs['units'].decode() + ")",
                   grid['lat'].attrs['standard_name'].decode() + " (" + grid['lat'].attrs['units'].decode() + ")",
                   "Precipitation (" + grid['precipitationCal'].attrs['units'].decode() + ")",]
dataset.head()

,longitude (degrees_east),latitude (degrees_north),Precipitation (mm/hr)
0,-179.949997,-89.949997,-9999.900391
1,-179.949997,-89.849998,-9999.900391
2,-179.949997,-89.750000,-9999.900391
3,-179.949997,-89.650002,-9999.900391
4,-179.949997,-89.550003,-9999.900391


In [36]:
np.amax(dataset['latitude (degrees_north)'])

89.95

In [38]:
dataset.to_csv('IMERG_20230907_1800Z.csv', index=False)

In [39]:
dataset = dataset[dataset['longitude (degrees_east)'] <= -44.7]
dataset = dataset[dataset['longitude (degrees_east)'] >= -56.8]
dataset = dataset[dataset['latitude (degrees_north)'] <= 22.7]
dataset = dataset[dataset['latitude (degrees_north)'] >= 10.6]
print(dataset.shape)
dataset.head()

(14641, 3)


,longitude (degrees_east),latitude (degrees_north),Precipitation (mm/hr)
2218606,-56.75,10.65,0.0
2218607,-56.75,10.75,0.0
2218608,-56.75,10.85,0.0
2218609,-56.75,10.95,0.0
2218610,-56.75,11.05,0.0


In [40]:
image = dataset.pivot(index='latitude (degrees_north)', columns='longitude (degrees_east)', values='Precipitation (mm/hr)')
image = image.T

In [41]:
image = image.iloc[:, :]
image.head()

latitude (degrees_north),10.650000,10.750000,10.850000,10.950000,11.050000,11.150000,11.250000,11.350000,11.450000,11.550000,...,21.750000,21.850000,21.950001,22.049999,22.150000,22.250000,22.350000,22.450001,22.549999,22.650000
longitude (degrees_east),,,,,,,,,,,,,,,,,,,,,
-56.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-56.650002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-56.549999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-56.450001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-56.349998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
image.to_csv('ATL_202313_C3_2023090718.csv', index=False, header = False)